In [3]:
# example of training an unstable gan for generating a handwritten digit
import tensorflow as tf
from os import makedirs
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.initializers import RandomNormal
from matplotlib import pyplot
 
# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1)):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# define model
	model = Sequential()
	# downsample to 14x14
	model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init, input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 7x7
	model.add(Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model
 
# define the standalone generator model
def define_generator(latent_dim):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# define model
	model = Sequential()
	# foundation for 7x7 image
	n_nodes = 128 * 7 * 7
	model.add(Dense(n_nodes, kernel_initializer=init, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((7, 7, 128)))
	# upsample to 14x14
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 28x28
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init))
	model.add(LeakyReLU(alpha=0.2))
	# output 28x28x1
	model.add(Conv2D(1, (7,7), activation='tanh', padding='same', kernel_initializer=init))
	return model
 
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
	# make weights in the discriminator not trainable
	discriminator.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(generator)
	# add the discriminator
	model.add(discriminator)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model
 
# load mnist images
def load_real_samples():
	# load dataset
	(trainX, trainy), (_, _) = load_data()
	# expand to 3d, e.g. add channels
	X = expand_dims(trainX, axis=-1)
	# select all of the examples for a given class
	selected_ix = trainy == 8
	X = X[selected_ix]
	# convert from ints to floats
	X = X.astype('float32')
	# scale from [0,255] to [-1,1]
	X = (X - 127.5) / 127.5
	return X
 
# # select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# select images
	X = dataset[ix]
	# generate class labels
	y = ones((n_samples, 1))
	return X, y
 
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = generator.predict(x_input)
	# create class labels
	y = zeros((n_samples, 1))
	return X, y
 
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, latent_dim, n_samples=100):
	# prepare fake examples
	X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
	# scale from [-1,1] to [0,1]
	X = (X + 1) / 2.0
	# plot images
	for i in range(10 * 10):
		# define subplot
		pyplot.subplot(10, 10, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(X[i, :, :, 0], cmap='gray_r')
	# save plot to file
	pyplot.savefig('results_convergence/generated_plot_%03d.png' % (step+1))
	pyplot.close()
	# save the generator model
	g_model.save('results_convergence/model_%03d.h5' % (step+1))
 
# create a line plot of loss for the gan and save to file
def plot_history(d_hist, g_hist, a_hist):
	# plot loss
	pyplot.subplot(2, 1, 1)
	pyplot.plot(d_hist, label='dis')
	pyplot.plot(g_hist, label='gen')
	pyplot.legend()
	# plot discriminator accuracy
	pyplot.subplot(2, 1, 2)
	pyplot.plot(a_hist, label='acc')
	pyplot.legend()
	# save plot to file
	pyplot.savefig('results_convergence/plot_line_plot_loss.png')
	pyplot.close()
 
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=128):
	# calculate the number of batches per epoch
	bat_per_epo = int(dataset.shape[0] / n_batch)
	# calculate the total iterations based on batch and epoch
	n_steps = bat_per_epo * n_epochs
	# calculate the number of samples in half a batch
	half_batch = int(n_batch / 2)
	# prepare lists for storing stats each iteration
	d_hist, g_hist, a_hist = list(), list(), list()
	# manually enumerate epochs
	for i in range(n_steps):
		# get randomly selected 'real' samples
		X_real, y_real = generate_real_samples(dataset, half_batch)
		# generate 'fake' examples
		X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
		# combine into one batch
		X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
		# update discriminator model weights
		d_loss, d_acc = d_model.train_on_batch(X, y)
		# prepare points in latent space as input for the generator
		X_gan = generate_latent_points(latent_dim, n_batch)
		# create inverted labels for the fake samples
		y_gan = ones((n_batch, 1))
		# update the generator via the discriminator's error
		g_loss = gan_model.train_on_batch(X_gan, y_gan)
		# summarize loss on this batch
		print('>%d, d=%.3f, g=%.3f, a=%d' % (i+1, d_loss, g_loss, int(100*d_acc)))
		# record history
		d_hist.append(d_loss)
		g_hist.append(g_loss)
		a_hist.append(d_acc)
		# evaluate the model performance every 'epoch'
		if (i+1) % bat_per_epo == 0:
			summarize_performance(i, g_model, latent_dim)
	plot_history(d_hist, g_hist, a_hist)
 
# make folder for results
makedirs('results_convergence', exist_ok=True)
# size of the latent space
latent_dim = 50
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# load image data
dataset = load_real_samples()
print(dataset.shape)
# train model
train(generator, discriminator, gan_model, dataset, latent_dim)

(5851, 28, 28, 1)
>1, d=0.699, g=0.693, a=14
>2, d=0.686, g=0.692, a=48
>3, d=0.674, g=0.691, a=50
>4, d=0.663, g=0.690, a=50
>5, d=0.652, g=0.686, a=50
>6, d=0.641, g=0.679, a=50
>7, d=0.632, g=0.668, a=50
>8, d=0.627, g=0.650, a=50
>9, d=0.625, g=0.627, a=50
>10, d=0.630, g=0.603, a=50
>11, d=0.639, g=0.591, a=50
>12, d=0.639, g=0.594, a=50
>13, d=0.632, g=0.612, a=50
>14, d=0.621, g=0.639, a=50
>15, d=0.610, g=0.671, a=50
>16, d=0.592, g=0.705, a=50
>17, d=0.576, g=0.738, a=100
>18, d=0.561, g=0.776, a=100
>19, d=0.547, g=0.818, a=100
>20, d=0.531, g=0.866, a=100
>21, d=0.513, g=0.917, a=99
>22, d=0.495, g=0.971, a=99
>23, d=0.462, g=1.037, a=99
>24, d=0.442, g=1.109, a=100
>25, d=0.410, g=1.189, a=100
>26, d=0.395, g=1.252, a=98
>27, d=0.368, g=1.320, a=99
>28, d=0.336, g=1.398, a=100
>29, d=0.333, g=1.441, a=98
>30, d=0.308, g=1.507, a=100
>31, d=0.265, g=1.613, a=100
>32, d=0.264, g=1.669, a=100
>33, d=0.245, g=1.722, a=98
>34, d=0.223, g=1.781, a=100
>35, d=0.204, g=1.868, a=99


>279, d=0.002, g=7.015, a=100
>280, d=0.001, g=6.960, a=100
>281, d=0.001, g=7.027, a=100
>282, d=0.001, g=7.060, a=100
>283, d=0.001, g=7.132, a=100
>284, d=0.001, g=7.097, a=100
>285, d=0.001, g=7.129, a=100
>286, d=0.001, g=7.185, a=100
>287, d=0.001, g=7.155, a=100
>288, d=0.001, g=7.109, a=100
>289, d=0.001, g=7.066, a=100
>290, d=0.000, g=7.130, a=100
>291, d=0.003, g=6.924, a=100
>292, d=0.001, g=6.892, a=100
>293, d=0.002, g=6.697, a=100
>294, d=0.001, g=6.663, a=100
>295, d=0.001, g=6.769, a=100
>296, d=0.001, g=6.910, a=100
>297, d=0.001, g=6.950, a=100
>298, d=0.001, g=6.972, a=100
>299, d=0.001, g=6.997, a=100
>300, d=0.001, g=7.049, a=100
>301, d=0.002, g=6.932, a=100
>302, d=0.001, g=6.949, a=100
>303, d=0.006, g=6.329, a=100
>304, d=0.004, g=5.723, a=100
>305, d=0.002, g=5.698, a=100
>306, d=0.002, g=6.117, a=100
>307, d=0.001, g=6.526, a=100
>308, d=0.001, g=6.772, a=100
>309, d=0.001, g=6.965, a=100
>310, d=0.001, g=7.101, a=100
>311, d=0.000, g=7.214, a=100
>312, d=0.